In [6]:
!pip install imageio av

     ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
     --------------------------------------- 0.0/25.3 MB 435.7 kB/s eta 0:00:59
     --------------------------------------- 0.1/25.3 MB 550.5 kB/s eta 0:00:46
     --------------------------------------- 0.1/25.3 MB 658.3 kB/s eta 0:00:39
     --------------------------------------- 0.1/25.3 MB 731.4 kB/s eta 0:00:35
     --------------------------------------- 0.1/25.3 MB 731.4 kB/s eta 0:00:35
     --------------------------------------- 0.1/25.3 MB 731.4 kB/s eta 0:00:35
     --------------------------------------- 0.2/25.3 MB 758.5 kB/s eta 0:00:34
     --------------------------------------- 0.2/25.3 MB 758.5 kB/s eta 0:00:34
      -------------------------------------- 0.3/25.3 MB 776.5 kB/s eta 0:00:33
      -------------------------------------- 0.4/25.3 MB 820.1 kB/s eta 0:00:31
      -------------------------------------- 0.5/25.3 MB 855.0 kB/s eta 0:00:30
      -------------------------------------- 0.5

In [2]:
import imageio.v3 as iio

im = iio.imread('imageio:chelsea.png')
print(im.shape)  # (300, 451, 3)

(300, 451, 3)


In [5]:
import torch
img = torch.from_numpy(im)
out = img.permute(2, 0, 1)

img.shape, out.shape

(torch.Size([300, 451, 3]), torch.Size([3, 300, 451]))

In [9]:
import imageio.v3 as iio

# read a single frame
frame = iio.imread(
    "imageio:cockatoo.mp4",
    index=42,
    plugin="pyav",
)

# bulk read all frames
# Warning: large videos will consume a lot of memory (RAM)
frames = iio.imread("imageio:cockatoo.mp4", plugin="pyav")

print(f"the len of frames: {len(frames)}")

the len of frames: 280


In [16]:
batch_size = len(frames)
batch = torch.zeros(batch_size, 3, frames[0].shape[0], frames[0].shape[1], dtype=torch.uint8)

for i, frame in enumerate(frames):
    frame_t = torch.from_numpy(frame)
    frame_t = frame_t.permute(2, 0, 1)
    frame_t = frame_t[:3]
    batch[i] = frame_t

batch

tensor([[[[116, 116, 116,  ...,  33,  33,  33],
          [116, 116, 116,  ...,  33,  33,  33],
          [116, 116, 116,  ...,  33,  33,  33],
          ...,
          [172, 175, 180,  ...,  48,  48,  48],
          [172, 175, 180,  ...,  48,  48,  48],
          [172, 175, 180,  ...,  48,  48,  48]],

         [[119, 119, 119,  ...,  29,  29,  29],
          [119, 119, 119,  ...,  29,  29,  29],
          [119, 119, 119,  ...,  29,  29,  29],
          ...,
          [166, 170, 175,  ...,  47,  47,  47],
          [166, 170, 175,  ...,  47,  47,  47],
          [166, 170, 175,  ...,  47,  47,  47]],

         [[104, 104, 104,  ...,  28,  28,  28],
          [104, 104, 104,  ...,  28,  28,  28],
          [104, 104, 104,  ...,  28,  28,  28],
          ...,
          [143, 147, 151,  ...,  52,  52,  52],
          [143, 147, 151,  ...,  52,  52,  52],
          [143, 147, 151,  ...,  52,  52,  52]]],


        [[[116, 116, 116,  ...,  33,  33,  33],
          [116, 116, 116,  ...,  33

In [17]:
batch = batch.float()
batch /= 255.0

n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std